# Example of relational database

![Title](diagrama.png)

**I take data from .csv file and fill my database.**

In [12]:
import pandas as pd
import numpy as np
import random

df=pd.read_csv ("data/SampleSuperstore.csv")
df.rename(columns={'Sub-Category': 'product'}, inplace=True)
df.head()

,Ship Mode,Segment,Country,City,State,Postal Code,Region,Category,product,Sales,Quantity,Discount,Profit
0,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,261.9600,2,0.00,41.9136
1,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,731.9400,3,0.00,219.5820
2,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,14.6200,2,0.00,6.8714
3,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,957.5775,5,0.45,-383.0310
4,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,22.3680,2,0.20,2.5164


**Create index for every table**

In [13]:
def put_id(col, col_id):
    name_id=col.lower()
    col_dict= {name_id:list(df[col].unique()),
         "id":list(range(1,len(df[col].unique())+1))}
    df[col_id]=df[col].apply(lambda x:col_dict.get("id")[col_dict.get(name_id).index(x)])
    
    df_col=pd.DataFrame(col_dict) #to return!
    print(col)
    print(df_col.head())
    print("**********************************************************")
    return df_col
    
df_category=put_id("Category","id_category")
df_city=put_id("City","id_city")
df_state=put_id("State","id_state")
df_segment=put_id("Segment","id_segment")
df_product=put_id("product","id_product") 

Category
          category  id
0        Furniture   1
1  Office Supplies   2
2       Technology   3
**********************************************************
City
              city  id
0        Henderson   1
1      Los Angeles   2
2  Fort Lauderdale   3
3          Concord   4
4          Seattle   5
**********************************************************
State
            state  id
0        Kentucky   1
1      California   2
2         Florida   3
3  North Carolina   4
4      Washington   5
**********************************************************
Segment
       segment  id
0     Consumer   1
1    Corporate   2
2  Home Office   3
**********************************************************
product
     product  id
0  Bookcases   1
1     Chairs   2
2     Labels   3
3     Tables   4
4    Storage   5
**********************************************************


In [15]:
df_product=df[["product","id_product","id_category"]].groupby("id_product").first().reset_index()
df_product.rename(columns={'id_product': 'id','product': 'product_name'}, inplace=True)
df_city[["id","city","id_state"]]=df[["City","id_city","id_state"]].groupby("id_city").first().reset_index()

Data for worker table

In [17]:
worker= {"id":[1,2,3,4,5,6],
         "name":["Juan", "Sara","Pilar", "Olga", "Pablo", "Sara"],
         "surname": ["Gonzalez", "Perez", "Fernandez", "Rodriguez", "Perez", "Lopez"]
        }
df_worker=pd.DataFrame(worker)
df["id_worker"]=0
df["id_worker"]=df["id_worker"].apply(lambda x:random.randint(1,6))
df_worker

,id,name,surname
0,1,Juan,Gonzalez
1,2,Sara,Perez
2,3,Pilar,Fernandez
3,4,Olga,Rodriguez
4,5,Pablo,Perez
5,6,Sara,Lopez


In [18]:
df_sale=df[['Sales', 'Quantity','id_city', 'id_segment','id_product', 'id_worker']]
df_sale.rename(columns={'Sales': 'sum','Quantity': 'quantity'}, inplace=True)
df_sale.head()

c:\users\usuario\appdata\local\programs\python\python39\lib\site-packages\pandas\core\frame.py:5034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,sum,quantity,id_city,id_segment,id_product,id_worker
0,261.9600,2,1,1,1,2
1,731.9400,3,1,1,2,1
2,14.6200,2,2,2,3,6
3,957.5775,5,3,1,4,5
4,22.3680,2,3,1,5,6


For every table de database I have dataframe with the same columns. I send them to BD.

In [23]:
from sqlalchemy import create_engine
from sqlalchemy import text
import pandas as pd
from datetime import date


host = "127.0.0.1:3306"
user="root"
passwd="1234"
database="trade"


engine = create_engine('mysql+mysqlconnector://' + user + ':' + passwd + '@' + host + '/' + database)

df_category.to_sql("category",con=engine, if_exists="append", index=False)
df_state.to_sql("state",con=engine, if_exists="append", index=False)
df_city.to_sql("city",con=engine, if_exists="append", index=False)
df_product.to_sql("product",con=engine, if_exists="append", index=False)
df_worker.to_sql("worker",con=engine, if_exists="append", index=False)
df_segment.to_sql("segment",con=engine, if_exists="append", index=False)
df_sale.to_sql("sale_product",con=engine, if_exists="append", index=False)